In [1]:
import grpc
from pkg.proto.aiservice.v1 import *
from internal.aiserver.chain import *
import nest_asyncio  # pip install nest_asyncio
import asyncio
import langchain

nest_asyncio.apply()  # 允许嵌套事件循环
async def generateTitleAndTagTest():
    async for response in stub.polishContent(PromptContentRequest(
            prompt="生成生动活泼的标题。",
            content="发达随父ijeasnvlanlreerlb4534哪控件那律哪里就可1"
        )):
        print("response:\n", response)

async def polishContentTest():
    async for response in stub.polishContent(PromptContentRequest(
            prompt="博客风格要求生动活泼。",
            content="发达随父ijeasnvlanlreerlb4534哪控件那律哪里就可1"
        )):
        print("response:\n", response.contentChunk)

async def summaryContentTest():
    async for response in stub.summaryContent(ContentRequest(
            content="发达随父ijeasnvlanlreerlb4534哪控件那律哪里就可1"
        )):
        print("response:\n", response.contentChunk)

async def searchPostsTest():
    response = await stub.searchPosts(PromptRequest(
            prompt="给我最让人震惊的一篇博客。",
        ))
    print("response:\n", response)

if __name__ == '__main__':
    channel = grpc.aio.insecure_channel("localhost:50051")
    stub:pb.AIServiceStub = pb.AIServiceStub(channel)

    asyncio.run(summaryContentTest())
    # response:PolishContentResponse = stub.polishContent(
    #     PromptContentRequest(
    #         prompt="博客风格要求生动活泼。",
    #         content="博客内容"
    #     )
    # )

response:
 
response:
 抱歉
response:
 ，
response:
 您提供的
response:
 博客内容
response:
 似乎包含
response:
 无
response:
 意义的字符
response:
 或乱
response:
 码，
response:
 无法生成
response:
 有效的摘要
response:
 或描述
response:
 。请
response:
 提供清晰
response:
 、有
response:
 意义的博客
response:
 内容，
response:
 我将为您
response:
 生成合适的
response:
 摘要。
response:
 


In [ ]:
from langgraph.prebuilt import create_react_agent

langgraph_agent_executor = create_react_agent(model, tools)


messages = langgraph_agent_executor.invoke({"messages": [("human", query)]})
{
    "input": query,
    "output": messages["messages"][-1].content,
}

In [ ]:

from langchain.callbacks.stdout import StdOutCallbackHandler
# from langchain.agents import AgentType, initialize_agent, load_tools, AgentExecutor, create_react_agent
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
from langchain.tools import StructuredTool
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough, RunnableBranch
from langchain.output_parsers import PydanticOutputParser
from pydantic import field_validator, BaseModel, Field
from typing import Dict, List, Any
from internal.pkg.runnable.llm import GetLanguageModel

def query():
    return "hello"
tools = []
tools.append(StructuredTool.from_function(
            func=query,
            name="查询数据库",
            description="输入查询语句（注意加分号），输出字符串作为结果"
        ))
agent = create_react_agent(
            GetLanguageModel(streaming=True),
            tools,
            debug=True,
        )

template = PromptTemplate(
            input_variables=["prompt", "memory"],
            template="""你是一个搜索助手，你需要根据用户的描述在数据库中搜索相关博客，返回相关博客的Id，并给出一个简要的搜索结果与用户需求贴合程度的分析。
            存储博客的数据库名称为post_table，其中包含title, content, postId, Id, updatedAt, createdAt等字段。
            注意，你只拥有数据库中表post_table的select权限，且每次搜索时必须限制搜索结果的数量最大为100，
            如非必要，不要搜索content字段，防止占用太多数据库资源。
            
            你可以进行多轮迭代使用工具来解决问题，当你决定继续获取信息时，在输出开头使用<continue>;当你决定输出结果给用户时，在输出开头使用<output>。在4轮迭代内需要返回最终结果。
            输出的格式为：<output><贴合程度分析>postid:postid:postid...\n
            输出的示例为：
            <example>
            输入：给我最让人震惊的一篇博客。
            输出：<output><你要求找到最能让你震惊的博客，但是没有说出哪类博客最能让你震惊，以下结果是一般而言能让人震惊的博客>34:43:56:88:204
            </example>
            用户的要求是：<request>{request}</request>\n
            当前操作的历史是：<memory>{memory}</memory>
            """,
        )
agent.invoke({"messages": {"role":"system", "content":template.invoke({"request":})}})